## Converting MP3 to WAV

In [2]:
from tqdm import tqdm
from pydub import AudioSegment
from os import listdir
from os.path import isfile, join

In [2]:
path_input = '../data/clean_audio'
path_output = '../data/clean_audio_WAV'

In [6]:
file_data = [f for f in listdir(path_input) if isfile (join(path_input, f))]
for line in tqdm(file_data):
    if ( line[-1:] == '\n' ):
        line = line[:-1]

    # Reading Song
    songname = path_input + '/' + line
    save_dest = path_output + '/' + line.split('.')[0] + '.wav'
    # print(save_dest)

    sound = AudioSegment.from_mp3(songname)
    sound.export(save_dest, format="wav")

100%|██████████| 10000/10000 [17:48<00:00,  9.35it/s]


## Converting WAV to Tensors

In [3]:
import os
import torch
import torch.nn.functional as F
import torchaudio

In [2]:
# UNCOMMENT EACH BLOCK FOR CLEAN AND NOISY DATASET

# # Settings for clean files
path_input = '../data/clean_audio_WAV'
path_output = '../data/clean_audio_tensors_v2'
FILE_HEADER = 'clean'

# # Settings for noisy files
# path_input = '../data/noisy_audio_WAV'
# path_output = '../data/noisy_audio_tensors_v2'
# FILE_HEADER = 'noisy'

In [ ]:
def pad(data, size):
    assert data.size(dim=1) <= size
    if data.size(dim=1) == size:
        return data
    padded_data  = F.pad(data, pad=(0, size - data.shape[1]))
    return padded_data  

In [5]:
file_data = [f for f in listdir(path_input) if isfile (join(path_input, f))]
for idx,line in enumerate(tqdm(file_data)):
    if ( line[-1:] == '\n' ):
        line = line[:-1]

    # Getting file path
    songname = path_input + '/' + line

    # Load .wav file
    waveform, sample_rate = torchaudio.load(songname)

    # Resample to 22050Hz
    resampler = torchaudio.transforms.Resample(sample_rate, 22050)
    resampled_waveform = resampler(waveform)

    # Ensure that tensors are in fixed shape of [1,22050]
    splits = list(torch.split(resampled_waveform, 22_050, dim=1))
    splits[-1] = pad(splits[-1], 22050)

    # Save each tensor
    for s_idx, split in enumerate(splits):
        fn = FILE_HEADER + str(idx) + '_' + str(s_idx) + '.pt'
        torch.save(split, os.path.join(path_output, fn))

100%|██████████| 10000/10000 [03:03<00:00, 54.38it/s]
